In [1]:
import pandas as pd
from functions import process_datasets

### 入出力データセットの名称
## 入力データ
train_file = 'train.csv'
test_file = 'test.csv'

### データのインポート
df_train = pd.read_csv('original_data/' + train_file)
df_test = pd.read_csv('original_data/' + test_file)
original_datasets = [df_train, df_test]

### データ加工の実施
model_input_datasets = []
model_input_datasets.append(process_datasets.pattern_1(*original_datasets))
model_input_datasets.append(process_datasets.pattern_2(*original_datasets))
model_input_datasets.append(process_datasets.pattern_3(*original_datasets))

In [2]:
## 準備
i = 2
model_name = 'newral_network'
response_variable_name = 'Survived'

x_train_val = model_input_datasets[i][0]
y_train_val = model_input_datasets[i][1]
x_test = model_input_datasets[i][2]
id_test = model_input_datasets[i][3] # for submitting result

## boolをintに変換する
x_train_val = process_datasets.bool_to_int(x_train_val)
x_test = process_datasets.bool_to_int(x_test)

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

/Users/keitaoikoshi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# from keras.layers import Dense, Dropout
# from keras.models import Sequential
# from sklearn.metrics import log_loss
# from sklearn.preprocessing import StandarScaler

# # データの分割
# from sklearn.model_selection import train_test_split
# # データを学習用、検証用に分割する(乱数シードを0、検証データの割合を0.3に指定)
# x_train,x_val,y_train,y_val = train_test_split(\
#     x_train_val, y_train_val, test_size=0.3, random_state=0\
#       )

# # データのスケーリング
# scaler = StandarScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)
# x_test = scaler.transform(x_test)

# # ニューラルネットモデルの構築
# model = Sequential()
# model.add()

In [ ]:
###  結果の出力
result_file_name = 'outputs/' + model_name + '_' + str(i+1) + '.csv'
df_result.to_csv(result_file_name, index=False)